# Setup


In [1]:
!make

CMake Error: The current CMakeCache.txt directory /home/gensenju/Documents/DL sys/dlsys-final/build/CMakeCache.txt is different than the directory /home/gensenju/Documents/DL sys/Needle/build where CMakeCache.txt was created. This may result in binaries being created in the wrong place. If you are not sure, reedit the CMakeCache.txt
CMake Error: The source "/home/gensenju/Documents/DL sys/dlsys-final/CMakeLists.txt" does not match the source "/home/gensenju/Documents/DL sys/Needle/CMakeLists.txt" used to generate cache.  Re-run cmake with a different source directory.
make: *** [Makefile:8: lib] Error 1


In [2]:
import sys
sys.path.append('./python')
import needle as ndl
from needle import nn
import numpy as np

## Создание тензоров

In [3]:
ndl.Tensor([[1, 2, 3], [3, 2, 1]])

needle.Tensor([[1 2 3]
 [3 2 1]])

In [4]:
ndl.Tensor(np.array([[1, 2, 3], [3, 2, 1]]))

needle.Tensor([[1 2 3]
 [3 2 1]])

## Операции над тензорами и autogradient

In [17]:
np.random.seed(42)

In [6]:
A = ndl.init.rand(3, 3)
A

needle.Tensor([[0.23378117 0.8887989  0.07809281]
 [0.0369595  0.97608066 0.22362131]
 [0.59372574 0.39184824 0.0528938 ]])

In [7]:
A.transpose()

needle.Tensor([[0.23378117 0.0369595  0.59372574]
 [0.8887989  0.97608066 0.39184824]
 [0.07809281 0.22362131 0.0528938 ]])

In [4]:
w1, A, w2 = ndl.Tensor([[1, 2]]), ndl.Tensor([[1, 2], [1, 2]]), ndl.Tensor([[1], [2]])
B = w1 @ A @ w2

In [5]:
B.inputs, B.op

((needle.Tensor([[3 6]]),
  needle.Tensor([[1]
   [2]])),
 <needle.ops.MatMul at 0x7febe8387ca0>)

In [10]:
B.backward() # вычисление всех градиентов

In [11]:
A.grad

needle.Tensor([[1 2]
 [2 4]])

## Создание слоев нейронных сетей и пример обучения модели

In [ ]:
linear_forward((10, 5), (1, 10)

In [19]:
f = ndl.nn.Linear(10, 5)
x = ndl.init.randn(1, 10)
f(x)

needle.Tensor([[ 0.894166    3.143468    0.7798127   0.01299441 -0.7340246 ]])

In [31]:
def get_tensor(*shape, entropy=1):
    np.random.seed(np.prod(shape) * len(shape) * entropy)
    return ndl.Tensor(np.random.randint(0, 100, size=shape) / 20, dtype="float32")

def get_int_tensor(*shape, low=0, high=10, entropy=1):
    np.random.seed(np.prod(shape) * len(shape) * entropy)
    return ndl.Tensor(np.random.randint(low, high, size=shape))

def learn_model_1d(feature_size, nclasses, _model, optimizer, epochs=1, **kwargs):
    np.random.seed(42)
    model = _model([])
    X = get_tensor(1024, feature_size).cached_data
    y = get_int_tensor(1024, low=0, high=nclasses).cached_data.astype(np.uint8)
    m = X.shape[0]
    batch = 32

    loss_func = ndl.nn.SoftmaxLoss()
    opt = optimizer(model.parameters(), **kwargs)

    for _ in range(epochs):
        for i, (X0, y0) in enumerate(zip(np.array_split(X, m//batch), np.array_split(y, m//batch))):
            opt.reset_grad()
            X0, y0 = ndl.Tensor(X0, dtype="float32"), ndl.Tensor(y0)
            out = model(X0)
            loss = loss_func(out, y0)
            loss.backward()
            # Opt should not change gradients.
            grad_before = model.parameters()[0].grad.detach().cached_data
            opt.step()
            grad_after = model.parameters()[0].grad.detach().cached_data
            np.testing.assert_allclose(grad_before, grad_after, rtol=1e-5, atol=1e-5, \
                                       err_msg="Optim should not modify gradients in place")


    return np.array(loss.cached_data)

In [32]:
learn_model_1d(64, 16, lambda z: ndl.nn.Sequential(nn.Linear(64, 32), nn.ReLU(), nn.Linear(32, 16)), ndl.optim.SGD, lr=0.01, momentum=0.0)
        

array(3.20700884)